In [6]:
import tarfile
import os 


import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data import random_split
import torchvision
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms


from src.utils_results import plot_img
from src.fedclass import Client
from src.utils_training import train_central, test_model
from src.utils_data import get_clients_data, data_preparation
from src.models import  GenericConvModel
from sklearn.model_selection import train_test_split



## Three Methods to load the dataset

In [11]:
### 1- Using raw image folder

dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
data_dir = './data/cifar10'

classes = os.listdir(data_dir + "/train")
dataset1 = ImageFolder(data_dir+'/train', transform=ToTensor())



### 2- Using project functions

dict_clients = get_clients_data(num_clients = 1, num_samples_by_label = 600, dataset = 'cifar10', nn_model = 'convolutional')
x_data, y_data = dict_clients[0]['x'], dict_clients[0]['y']
x_data = np.transpose(x_data, (0, 3, 1, 2))

dataset2 = TensorDataset(torch.tensor(x_data, dtype=torch.float32), torch.tensor(y_data, dtype=torch.long))



### 3 - Using CIFAR10 dataset from Pytorch

cifar10 = torchvision.datasets.CIFAR10("datasets", download=True, transform=ToTensor())
(x_data, y_data) = cifar10.data, cifar10.targets
x_data = np.transpose(x_data, (0, 3, 1, 2))
dataset3 = TensorDataset(torch.tensor(x_data, dtype=torch.float32), torch.tensor(y_data, dtype=torch.long))


Using downloaded and verified file: ./cifar10.tgz


/tmp/ipykernel_6044/2990241823.py:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path='./data')


Files already downloaded and verified
Files already downloaded and verified


In [13]:
model = GenericConvModel(32,3)

## Conversion to dataloaders

In [14]:

random_seed = 42
torch.manual_seed(random_seed);
val_size = 5000
train_size = len(dataset1) - val_size

train_ds, val_ds = random_split(dataset1, [train_size, val_size])

batch_size=128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)


In [15]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.001

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=opt_func):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history


In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 1.7809, val_loss: 1.4422, val_acc: 0.4745
Epoch [1], train_loss: 1.2344, val_loss: 1.0952, val_acc: 0.6092
Epoch [2], train_loss: 0.9971, val_loss: 0.9526, val_acc: 0.6552
Epoch [3], train_loss: 0.8338, val_loss: 0.8339, val_acc: 0.7085
Epoch [4], train_loss: 0.7093, val_loss: 0.7892, val_acc: 0.7239
Epoch [5], train_loss: 0.6082, val_loss: 0.7572, val_acc: 0.7490


In [25]:
result = evaluate(model, test_loader)

In [26]:
result

{'val_loss': 2.3049447536468506, 'val_acc': 0.10572139918804169}